In [1]:
import random 
from typing import Optional
import matplotlib.animation as animation
from matplotlib.animation import FuncAnimation
import matplotlib.pyplot as plt
from scipy import stats
import numpy as np

import pygame
import sys

pygame 2.5.1 (SDL 2.28.2, Python 3.9.13)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
class Agent(pygame.sprite.Sprite):

    def __init__(self):
        self.position = 0                                             #posición inicial, todos los autos entran al inicio de la autopista
        self.max_velocity = 80/3.6                                    #velocidad maxima, todos los autos tienen la misma velocidad maxima (sabemos que en la Gral. Paz es de 80 km/h)  
        self.length = 5                                               #longitud del vehiculo, todos los autos tienen la misma longitud 
        self.front_agent = None                                       #agente de adelante
        self.back_agent = None                                        #agente de atras
        
        '''
        velocidad de agente es generada a partir de la distribución triangular con los siguientes parametros:
        - 30,000 m/s: velocidad minima
        - 100,000 m/s: velocidad maxima
        - 70,000 m/s: moda
        '''
        self.velocity = random.triangular(30/3.6, 100/3.6, 70/3.6)
        self.time = 0
        self.arrival_time = 0
    
        self.reaction_time = random.normalvariate(1, 0.2) 
    
    def __str__(self):
        front_agent_info = (
            f"(t={self.front_agent.time}, x={self.front_agent.position}, "
            f"v={self.front_agent.velocity})"
            if self.front_agent is not None else "None"
        )
        
        back_agent_info = (
            f"(t={self.back_agent.time}, x={self.back_agent.position}, "
            f"v={self.back_agent.velocity})"
            if self.back_agent is not None else "None"
        )
        
        return (
            f"Agent(t={self.time}, at={self.arrival_time}, x={self.position}, "
            f"v={self.velocity}, rt={self.reaction_time}, "
            f"fa={front_agent_info}, ba={back_agent_info})"
        )

    def dist_to_front_agent(self):
        if self.front_agent == None:
            return None

        else: #hay otro agente adelante
            return abs(self.position - self.front_agent.position - self.length)


    def dist_to_back_agent(self):
        if self.back_agent == None:
            return None

        else: #hay otro agent atras
            return abs(self.back_agent.position - self.position - self.back_agent.length)

    def speed_variation(self):
        if self.dist_to_front_agent() == None or self.dist_to_front_agent() >= 20:
            self.velocity = random.normalvariate(self.velocity, 5/3.6)


    def reduce_speed(self):
        # si se encuentra "cerca" del agente de adelante, reducirá su velocidad mediante una distribución exponencial con media 1.
        if self.dist_to_front_agent() is not None and self.dist_to_front_agent() < 10:
            reduction = (self.velocity - self.front_agent.velocity) / (1 - stats.expon.rvs(1/1))
            self.velocity -= reduction  # Reducir la velocidad
            
        # si se encuentra "lejos" del agente de adelante, reducirá su velocidad mediante una distribución exponencial con media 3.
        elif self.dist_to_front_agent() is not None and 10 <= self.dist_to_front_agent() <= 20:
            reduction = (self.velocity - self.front_agent.velocity) /(1 - stats.expon.rvs(1/3))
            self.velocity -= reduction  # Reducir la velocidad


    def move(self):
        self.position = self.position + self.velocity * 1 
    
    def get_position(self):
        return self.position

    def update(self):
        self.move()
        self.reduce_speed()
        self.speed_variation()

In [3]:
class Road:
    
    def __init__(self, m:int, c:bool):
        self.km = m
        self.agents = []
        self.time = 0
        self.total_time = 700
        self.congestion = c
    
    def add_agent(self, rate):
        # Si no hay agentes en la Gral.Paz, agregamos el primer agente
        if not self.agents:
            print("Agregando primer agente...")
            self.agents.append(Agent())
            print("Primer agente agregado: " + str(self.agents[0]))
            
            self.time_to_next_arrival = stats.expon.rvs(scale=1/rate)

        else:
            # Verificamos el tiempo desde el último agente
            if self.time - self.agents[-1].arrival_time > self.time_to_next_arrival:
                print("Agregando agente nro. ", len(self.agents))
                # Agregamos un nuevo agente a la lista
                new_agent = Agent()
                self.agents.append(new_agent)
                new_agent.arrival_time = self.time
                self.time_to_next_arrival = stats.expon.rvs(scale=1/rate)
                
                if len(self.agents) == 2:
                    position_new_agent = self.agents.index(new_agent)
                    new_agent.front_agent = self.agents[position_new_agent - 1]
                    self.agents[position_new_agent - 1].back_agent = new_agent

                # Configuramos el front y back car para el nuevo agente
                elif len(self.agents) >= 2:
                    position_new_agent = self.agents.index(new_agent)
                    new_agent.front_agent = self.agents[position_new_agent - 1]
                    self.agents[position_new_agent - 1].back_agent = new_agent

        self.update_road()

    def time_lapse(self):
        while self.time < self.total_time:
            if self.congestion == True:
                # Agrega autos con distribución exponencial negativa con media = 15
                self.add_agent(15)
            else:
                # Agrega autos con distribución exponencial negativa con media = 45
                self.add_agent(45)
        
    
    def update_road(self):
        self.time += 1
        for agent in self.agents:
            agent.update()
            print("Estado agente nro:", self.agents.index(agent), "->", agent)
            # print("Posición del agente:", self.agents.index(agent), "->", agent.get_position())
            
            #Si el agente llega al final de la carretera lo eliminamos
            if agent.get_position() > self.km:
                self.agents.remove(agent)

            #Reasignamos el front y back car de los agentes extremos de la lista
            if len(self.agents) > 0:
                self.agents[0].front_agent = None

        self.time_lapse()
        
    def plot(self):
        positions = []  # Lista para almacenar las posiciones de los agentes
        for agent in self.agents:
            positions.append(agent.get_position())  # Almacena la posición del agente en la lista

        return positions  # Devuelve la lista de posiciones para la animación

In [ ]:
road = Road(m=25000, c=True)
road.time_lapse()

In [5]:
import pygame
import sys

# Inicializar Pygame
pygame.init()

# Configuración de la ventana
width, height = 1000, 200
screen = pygame.display.set_mode((width, height))
pygame.display.set_caption("Avenida General Paz")

# Colores
white = (255, 255, 255)
gray = (200, 200, 200)
green = (0, 150, 0)
black = (0, 0, 0)

# Crear una carretera
general_paz = Road(1000, True)

# Bucle principal
running = True
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    # Actualizar la carretera y los agentes en cada iteración
    general_paz.update_road()

   # Dibujar el pasto verde arriba de la carretera
    pygame.draw.rect(screen, green, (0, 0, width, height // 3))

    # Carretera gris
    pygame.draw.rect(screen, gray, (0, height // 3, width, height // 3))

    # Dibujar el pasto verde abajo de la carretera
    pygame.draw.rect(screen, green, (0, 2 * height // 3, width, height // 3))


    # Líneas blancas en el medio de la carretera
    line_width = 10
    line_height = 5
    num_lines = 30
    gap = width // (num_lines + 1)
    for i in range(num_lines):
        x = (i + 1) * gap - line_width // 2
        pygame.draw.rect(screen, white, (x, height // 2 - line_height // 2, line_width, line_height))


# Dibujar los agentes en la carretera
    for agent in general_paz.agents:
        pygame.draw.rect(screen, black, (agent.get_position(), height // 3 - 10, agent.length, 20))

    pygame.display.flip()

# Salir del juego
pygame.quit()
sys.exit()
